In [18]:
# imports
from dash import Dash, html, dcc
import pandas as pd
import numpy as np
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate
from jupyter_dash import JupyterDash
from plotly.offline import init_notebook_mode
import plotly.express as px

# Ski Resort Dashboard for North America

### Objective

- Build two working dash applications
- multiple chart types + interactive elements
- callback functions with multiple inputs and outputs

##### Charts

- density mapbox with color intensity based on total slopes:
  - price slider to filter out resorts that are too cheap/expencive
  - radio toggle that filters between with and without night skiing
  - dynamic title based on the price
<br></br>
  
  
  
- Top 10 Resorts Based off a selected dataframe column:
  - checklist for canada, us or both
  - with and without night skiing radio button
  - interactive element that sorts visual in ascending or descending
  - visual title == dynamic based off column selected
  - toggle between mean or median of said column
  


In [4]:
# Set the default color scale to white-blue
px.defaults.color_continuous_scale = ["white", "blue"]

In [29]:
url = r"/Users/chrismembrey/Desktop/DS/projects/plotly_dash_course/Course_Materials/Data/Ski Resorts/resorts.csv"
am_ski = pd.read_csv(url, encoding = "ISO-8859-1").query("Continent == 'North America'")
am_ski.head()

,ID,Resort,Latitude,Longitude,Country,Continent,Price,Season,Highest point,Lowest point,...,Snow cannons,Surface lifts,Chair lifts,Gondola lifts,Total lifts,Lift capacity,Child friendly,Snowparks,Nightskiing,Summer skiing
3,4,Red Mountain Resort-Rossland,49.105520,-117.846280,Canada,North America,60,December - April,2075,1185,...,0,2,5,1,8,9200,Yes,Yes,Yes,No
10,11,Fernie,49.504175,-115.062867,Canada,North America,67,December - April,2134,1052,...,11,3,7,0,10,14514,Yes,Yes,No,No
11,12,Sun Peaks,50.884468,-119.882329,Canada,North America,62,November - April,2082,1198,...,0,6,6,0,12,13895,Yes,Yes,Yes,No
12,13,Panorama,50.736999,-119.120561,Canada,North America,62,December - April,2365,1140,...,0,3,6,4,13,11890,Yes,Yes,Yes,No
21,22,Steamboat,35.754022,-109.853751,United States,North America,120,November - April,3221,2103,...,0,1,14,2,17,32720,Yes,Yes,Yes,No


In [56]:
app = JupyterDash(__name__)


app.layout = html.Div([
    
    html.H1("North American Ski Resorts"),
    
    dcc.Slider(
        id='price',
        min=am_ski['Price'].min(),
        max=am_ski['Price'].max(),
        step=10,
        value=am_ski['Price'].max(),
        marks={i: f'${i}' for i in range(am_ski['Price'].min(), 
                                         am_ski['Price'].max()+1, 10)}

              ),
    
    dcc.Checklist(
    
        id='night-ski',
        options = ['Night Ski Only'],
        value = []
    ),
    
    dcc.Graph(id='density-mapbox-total-slopes'),
    
    dcc.Dropdown(
    
        id='country-dropdown',
        options= ['Canada' , 'United States'],
        value = ['Canada' , 'United States'],
        multi = True,
        style={'margin-bottom': '2rem'}
    ),
    
    
    dcc.RadioItems(
    
        id='desc-asc',
        options = ['Ascending', 'Descending'],
        value = 'Descending',
        inline = True,
        style={'margin-bottom': '2rem'}
    ),
    
    dcc.Dropdown(
        id='field',
        options = am_ski.select_dtypes(include = 'number').columns,
        value = 'Price',
        style={'margin-bottom': '2rem'}
    
    ),
    
    dcc.Graph(id='top-10-graph')
    
    
])



@app.callback(

    Output('density-mapbox-total-slopes' , 'figure'),
    Output('top-10-graph', 'figure'),
    Input('price', 'value'),
    Input('night-ski', 'value'),
    Input('desc-asc', 'value'),
    Input('country-dropdown', 'value'),
    Input('field', 'value')
)
def create_app(price , night_ski, desc_asc , 
               country_dropdown, chosen_field):
    
    # deal with price
    am_ski_price = am_ski.query(f"Price <= @price")
    
    # deal with night skiing
    if len(night_ski) == 1:
        
        am_ski_price = am_ski_price.query("Nightskiing == 'Yes'")
        
    
    fig_mapbox = px.density_mapbox(
    
        am_ski_price,
        lat="Latitude",
        lon="Longitude",
        z='Total lifts',
        title=f'Density of Lifts based on Resorts less than {price}$',
        mapbox_style="stamen-terrain",
        center={"lat": 44.5, "lon": -103.5},
        zoom = 1.5,
        hover_data = 'Resort'
        
    )
    
    
    am_ski_price_country = (am_ski_price
                            .query("Country in @country_dropdown"))
    
    # top 10 resorts for the chosen numerical field
    am_ski_price_country_top_10 = (
    
        am_ski_price_country
        .nlargest(10 , chosen_field)
    )
    

    
### issue with doing it the below way was the resorts were separated by
### country and then sorted
#     # deal with ascending or descending
#     if desc_asc == 'Descending':
#         am_ski_price_country_top_10 = (am_ski_price_country_top_10
#                                        .sort_values(by = chosen_field, ascending = False)
#                                        )
        
#     else:
#         am_ski_price_country_top_10 = (am_ski_price_country_top_10
#                                        .sort_values(by = chosen_field, ascending = True)
#                                        )
        
        
    
    
    fig_bar = px.bar(
    
        am_ski_price_country_top_10,
        x=chosen_field,
        y='Resort',
        color='Country',
        title=f"Top 10 Resorts based on {chosen_field}",
        
        
    )
    
    # specify the order of the resorts on the y-axis
    # we do the ascending and descending this way to prevent the issue
    # above
    if desc_asc == 'Descending':
        
        fig_bar.update_layout(yaxis={'categoryorder':'total descending'})

    else:
        fig_bar.update_layout(yaxis={'categoryorder':'total ascending'})
        
    
    
    return fig_mapbox, fig_bar

if __name__ == '__main__':
    
    app.run_server(mode='inline', debug = True, port = 9989)

Dash is running on http://127.0.0.1:9989/

